In [1]:
'''
Set path to the folder of used python files
'''
import sys
import pandas as pd
import numpy as np
sys.path.insert(0, '/Users/lynnjiang/liuGit/Pybook/Clinic-Analysis/Scripts/Data_Cleaning')
from Abstract import Abstract

import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
import time

In [2]:
aa = Abstract()

In [10]:
data_name = 'procedures'
user_vec = aa.read_data('temp/USER_VECTORS/%s_uservectors' % data_name)
user_vec.head()

,Unnamed: 0,10,11,12,13,14,15,16,17,18,...,9977,9978,9979,9981,9982,9983,9984,9988,9995,9999
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Rename the first coloumn
# user_vec.rename(columns={'Unnamed: 0':'SUBJECT_ID'}, inplace=True)

In [12]:
## Get 10 randomly selected sub datasets
for i in range(10):
    aa.write2file(user_vec.sample(frac=0.05), 'USER_VECTORS/%s_sample%d'% (data_name,i))
# user_vec.columns

## Cluster

In [41]:
from collections import Counter
from scipy.stats import ks_2samp, chisquare
import sklearn.cluster as cluster
import time
def cluster_alg(data, alg, args, kwds):
    start_time = time.time()
    labels = alg(*args, **kwds).fit_predict(data)
    end_time = time.time()
    labels_count = Counter(labels)
    print("Time spent: %.2f s \nClusters: %s \n " %(end_time-start_time, labels_count))
    return labels, labels_count

In [42]:
## return comparising results of two distributions using ks2samp
def ks2samp(col, arraya, arrayb):
    result = ks_2samp(arraya, arrayb)
    return col, result.statistic, result.pvalue


def get_cluster_stats(alg, params, data_name):
    
    all_stats_df = pd.DataFrame()
    all_labels_count = []
    for i in range(10):
        sample_i = aa.read_data('temp/USER_VECTORS/%s_sample%d' % (data_name,i))
        ## Specific cluster algorithm
        labels, labels_count = cluster_alg(sample_i.iloc[:,1:], alg, (), params)
        all_labels_count.append(list(labels_count.values()))
        label_values = list(set(labels))
        pred_df = pd.concat([sample_i.iloc[:,1:], pd.Series(labels)], axis=1)
    
        ## Split feature data into two parts based on predicted labels for analyzing distributions from two parts for each feature
        pred_0_df, pred_1_df = [x for _, x in pred_df.groupby(pred_df[0] == label_values[0])]
        columns = sample_i.columns[1:]

        cluster_stas_list = list(map(lambda col: ks2samp(col,pred_0_df[col], pred_1_df[col]), columns))
        cluster_stats_df = pd.DataFrame(cluster_stas_list, columns=['CODE', 'STATISTIC', 'PVALUE'])
        all_stats_df = all_stats_df.append(cluster_stats_df)
    
    # Get average statistics of all sub dataset
    pure_stats_df = all_stats_df.groupby('CODE')['STATISTIC','PVALUE'].agg('mean')
    pure_stats_df = pure_stats_df.reset_index().sort_values(by='PVALUE')
    aa.write2file(pure_stats_df,'CLUSTER_STATISTICS/%s_%s_ks2' % (data_name,alg.__name__))
    avg_labels_count = dict(pd.DataFrame(all_labels_count).mean())
    print('Cluster counts: %s\nNumbers of features with p<0.05: %d' % (avg_labels_count, len(pure_stats_df[pure_stats_df['PVALUE']<=0.05])))
    return pure_stats_df

    

In [39]:
## test one time
# alg = cluster.SpectralClustering
# params = {'n_clusters':2, 'affinity': 'rbf'}
# sample_i = aa.read_data('temp/USER_VECTORS/%s_sample%d' % ('diag',0))
# labels = cluster_alg(sample_i.iloc[:,1:], alg, (), params)


# def test(data_name='diag',alg=cluster.DBSCAN,params={'eps':10}):
def test(data_name='prescriptions',alg=cluster.DBSCAN,params={'eps':7}):
    sample_i = aa.read_data('temp/USER_VECTORS/%s_sample%d' % (data_name,1))
    cluster_alg(sample_i.iloc[:,1:], alg, (), params)
#     return sample_i
  
test()

## test code
# for i in range(10,110,10):
#     test(params={'eps':i})

Total number of features: 1968 
Time spent: 30.51 s 
Clusters: Counter({-1: 1937, 1: 15, 0: 9, 2: 7}) 
 


In [46]:
## Get all statistics
## AgglomerativeClustering,{'n_clusters':2, 'linkage':'ward'}
data_name = 'procedures'
cluster_stats = get_cluster_stats(cluster.KMeans, {'n_clusters':2}, data_name)
len(cluster_stats)


Time spent: 1.82 s 
Clusters: Counter({0: 1504, 1: 607}) 
 
Time spent: 1.49 s 
Clusters: Counter({0: 1526, 1: 585}) 
 
Time spent: 1.77 s 
Clusters: Counter({1: 1511, 0: 600}) 
 
Time spent: 1.52 s 
Clusters: Counter({0: 1539, 1: 572}) 
 
Time spent: 1.78 s 
Clusters: Counter({0: 1543, 1: 568}) 
 
Time spent: 2.25 s 
Clusters: Counter({0: 1534, 1: 577}) 
 
Time spent: 2.00 s 
Clusters: Counter({1: 1486, 0: 625}) 
 
Time spent: 1.85 s 
Clusters: Counter({0: 1497, 1: 614}) 
 
Time spent: 1.77 s 
Clusters: Counter({0: 1520, 1: 591}) 
 
Time spent: 1.73 s 
Clusters: Counter({0: 1489, 1: 622}) 
 
Cluster counts: {0: 1233.6, 1: 877.4}
Numbers of features with p<0.05: 17


2009

In [ ]:
len(cluster_stats[cluster_stats['PVALUE']<=0.05])

In [ ]:
# all_stats_df.head()
pure_stats_df = all_stats_df.groupby('CODE')['STATISTIC','PVALUE'].agg('mean')
pure_stats_df.head()

In [ ]:
# pure_stats_df = pure_stats_df.reset_index()
# pure_stats_df.head()
aa.write2file(pure_stats_df.sort_values(by='PVALUE'),'CLUSTER_STATISTICS/diag_kmeans_ks2')

In [ ]:
## Kmeans
diag_labels = cluster_alg(dia_sample_0.iloc[:,1:], cluster.KMeans, (), {'n_clusters':2})
diag_pred_df = pd.concat([dia_sample_0.iloc[:,1:], pd.Series(diag_labels)], axis=1)
diag_pred_df.head()

In [ ]:
## Split feature data into two parts based on predicted labels for analyzing distributions from two parts for each feature
diag_pred0_df, diag_pred1_df = [x for _, x in diag_pred_df.groupby(diag_pred_df[0] == 1)]
diag_pred0_df.head()

In [ ]:
## return comparising results of two distributions using ks2samp
def ks2samp(col, arraya, arrayb):
    result = ks_2samp(arraya, arrayb)
    return col, result.statistic, result.pvalue

In [ ]:
diag_columns = dia_sample_0.iloc[:,1:].columns
# result = (ks_2samp(diag_pred0_df[diag_columns[0]],diag_pred1_df[diag_columns[0]]))
diag_ks2samp_list = list(map(lambda col: ks2samp(col,diag_pred0_df[col], diag_pred1_df[col]), diag_columns))
diag_ks2samp_list

In [ ]:
diag_cluster_stats= pd.DataFrame(diag_ks2samp_list, columns=['DIAG_IDC9_CODE', 'STATISTIC', 'PVALE'])
diag_cluster_stats.head()

In [ ]:
chisquare([2,3,11,0,2],[2,5,1])

In [ ]:
diag_cluster_stats.sort_values(by='PVALE')[:20]

In [ ]:

np.random.seed(123456)
x = np.random.normal(0, 1, 1000)
y = np.random.normal(0, 1, 800)
print('a', ks_2samp(x, y))
# print('b', ks_2samp(y,x))

# print(chisquare(x, f_exp=y))



In [ ]:
from sklearn.decomposition import PCA
import umap


In [ ]:
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}
## Get clusters
def plot_clusters(data, algorithm, args, kwds):
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]

## UMP transform
#     ump_embed = umap.UMAP().fit_transform(data)
## PCA transform    
    pca = PCA(n_components=2).fit(data)
    pca_2d = pca.transform(data)
    
    plt.scatter(pca_2d[:,0], pca_2d[:,1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    print('Clustering took {:.2f} s'.format(end_time - start_time))
    print(('Cluster counter: %s') % Counter(labels))
    aa.write2file(pd.DataFrame({'LABEL': labels}),'CLUSTERS/lab_%s' % str(algorithm.__name__))

In [ ]:
plot_clusters(user_vec.iloc[:,1:], cluster.KMeans, (), {'n_clusters':3})

In [ ]:
plot_clusters(user_vec.iloc[:,1:], cluster.AffinityPropagation, (), {'preference':-5.0, 'damping':0.95})

## Reduce dimensionality

In [ ]:
import umap
def ump(data):
    
    ump_embed = umap.UMAP().fit_transform(data)
    plt.figure('ump')
    plt.scatter(ump_embed[:,0], ump_embed[:,1])
    plt.show()
    
# from sklearn.datasets import load_digits

# digits = load_digits()

embedding = umap.UMAP().fit_transform(user_finaldf)

actual_labels['LABEL'] = actual_labels['LABEL'].astype(int)
actual_labels['PREDICTED']=y_pred
actual_labels['X0']=embedding[:,0]
actual_labels['X1']=embedding[:,1]
# actual_labels[:10]
# pd.concat([y_pred, actual_labels['LABEL'].astype(int)],axis=1).head()
# embedding.tolist()
true_X = actual_labels[actual_labels['LABEL']==0]
false_X = actual_labels[actual_labels['LABEL']==1]